
## Pulling data from Deribit 



In [1]:
import requests
import json
import pandas as pd 
import schedule
import time
import pyodbc


In [2]:
conn= pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                    'Server=BADRI-180;'
                    'Database=combine;'
                    'Trusted_Connection=yes;')

In [3]:
link='https://www.deribit.com/api/v2/public/get_book_summary_by_currency?currency=BTC&currency=ETH&kind=option'
cursor=conn.cursor()


In [4]:
def write_to_excel():
    response_API=requests.get(link)
    data=response_API.text
    parse_json= json.loads(data)
    
    currency=parse_json['result']
    #print("currency = ", currency)
    multiple_level_data=pd.json_normalize(parse_json,record_path=['result'])
    multiple_level_data['date1']=pd.to_datetime(multiple_level_data['creation_timestamp'],unit='ms').dt.date
    multiple_level_data['time1']=pd.to_datetime(multiple_level_data['creation_timestamp'],unit='ms').dt.time
    #multiple_level_data['time1']+=pd.Timedelta(hours=5)
    for index, row in multiple_level_data.iterrows():
        cursor.execute("INSERT INTO Combine.dbo.Options_API ([date1],[time1], [volume],[btc_price],[open_interest],[mark_price],[instrument_name]) values(?,?,?,?,?,?,?)", row.date1, row.time1, row.volume, row.underlying_price, row.open_interest, row.mark_price, row.instrument_name)
             
schedule.every(1).minutes.do(write_to_excel)
while True:
    schedule.run_pending()
    time.sleep(1)
    

KeyboardInterrupt: 

In [5]:
conn.commit()
cursor.close()